In [ ]:
!git clone -q https://github.com/OpenAccess-AI-Collective/axolotl
%cd axolotl
!pip install -qqq packaging huggingface_hub torch==2.1.1 --progress-bar off
!pip install -qqq -e '.[flash-attn,deepspeed]' --progress-bar off
!pip install -qqq mlflow

/content/axolotl
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu121 requires torch==2.1.0, but you have torch 2.1.1 which is incompatible.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 2.1.1 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 2.1.1 which is incompatible.
torchvision 0.16.0+cu121 requires torch==2.1.0, but you have torch 2.1.1 which is incompatible.
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchdata 0.7.0 requir

In [ ]:
# from transformers import (
#     AutoModelForCausalLM,
#     AutoTokenizer,
# )

# tokenizer = AutoTokenizer.from_pretrained("yanolja/KoSOLAR-10.7B-v0.2", eos_token='<|im_end|>', trust_remote_code=True)
# print(tokenizer.__class__.__name__)

In [ ]:
import yaml

new_model = "yanolja-axolotl-aug-noise"
yaml_string = """
base_model: yanolja/KoSOLAR-10.7B-v0.2
model_type: LlamaForCausalLM
tokenizer_type: LlamaTokenizerFast
is_llama_derived_model: true

load_in_8bit: false
load_in_4bit: true
strict: false

datasets:
  - path: /content/drive/MyDrive/DACON/Hansol_QA/datas/aug_noise_train.jsonl
    ds_type: json
    type:
        system_prompt: "-친절하게 답변하는 도배하자 처리 전문가입니다.
                        -설명은 이해하기 쉽고 명료하게 합니다.
                        -건축구조, 마감재, 마감하자, 시공, 인테리어,타 마감하자,기타 측면에서 대답합니다.
                        -그 외의 것들에 대한 질문에는 '잘 모르겠습니다'라고 답변합니다.
                        "
        system_format: "{system}"
        field_system: system
        field_instruction: question
        field_input: ""
        field_output: answer

    format: |-
        User: {instruction} {question}
        Assistant: {answer}
    no_input_format: "{instruction} {question} "

dataset_prepared_path:
val_set_size: 0.05
output_dir: /content/drive/MyDrive/DACON/Hansol_QA/models/yanolja-axolotl-aug-noise-sysprompt

adapter: qlora

lora_model_dir:



sequence_len: 1096
sample_packing: true
pad_to_sequence_len: true

lora_r: 32
lora_alpha: 16
lora_dropout: 0.05
lora_target_modules:
lora_target_linear: true
lora_fan_in_fan_out:
lora_on_cpu: true

special_tokens:
  # bos_token: "<s>"
  # eos_token: "<|im-end|>"
  # unk_token: "<unk>"

wandb_mode: offline

gradient_accumulation_steps: 2
micro_batch_size: 4
num_epochs: 3
# max_steps: 20
optimizer: paged_adamw_32bit
lr_scheduler: cosine
learning_rate: 2e-4

train_on_inputs: false
group_by_length: false
bf16: false
fp16: true
tf32: false

gradient_checkpointing: true
early_stopping_patience:
resume_from_checkpoint:
local_rank:
logging_steps: 1
xformers_attention:
flash_attention: false

warmup_steps: 10
evals_per_epoch:
saves_per_epoch: 1
debug:
deepspeed:
weight_decay: 0.001
fsdp:
fsdp_config:
special_tokens:

"""
#/content/drive/MyDrive/DACON/Hansol_QA/model/yanolja-axolotl
# Convert the YAML string to a Python dictionary
yaml_dict = yaml.safe_load(yaml_string)

# Specify your file path
yaml_file = 'config.yaml'

# Write the YAML file
with open(yaml_file, 'w') as file:
    yaml.dump(yaml_dict, file)

In [ ]:
!accelerate launch -m axolotl.cli.train config.yaml

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
[2024-03-07 19:08:47,405] [INFO] [datasets.<module>:58] [PID:48199] PyTorch version 2.1.1 available.
[2024-03-07 19:08:47,406] [INFO] [datasets.<module>:95] [PID:48199] TensorFlow version 2.15.0 available.
[2024-03-07 19:08:47,407] [INFO] [datasets.<module>:108] [PID:48199] JAX version 0.4.23 available.
2024-03-07 19:08:48.992343: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-07 19:08:48.992391: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to 

In [ ]:
from google.colab import runtime
import time

time.sleep(600)
runtime.unassign()

In [ ]:
!python3 -m axolotl.cli.merge_lora config.yaml --lora_model_dir="/content/drive/MyDrive/DACON/Hansol_QA/models/yanolja-axolotl-aug-noise-sysprompt"

/usr/local/lib/python3.10/dist-packages/bitsandbytes/cextension.py:31: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "
/usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
[2024-03-08 07:14:43,701] [INFO] [numexpr.utils._init_num_threads:160] [PID:2300] NumExpr defaulting to 8 threads.
[2024-03-08 07:14:44,046] [INFO] [datasets.<module>:58] [PID:2300] PyTorch version 2.1.1 available.
[2024-03-08 07:14:44,047] [INFO] [datasets.<module>:95] [PID:2300] TensorFlow version 2.15.0 available.
[2024-03-08 07:14:44,048] [INFO] [datasets.<module>:108] [PID:2300] JAX version 0.4.23 available.
2024-03-08 07:14:46.225684: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register fa

In [ ]:
!python -m axolotl.cli.inference config.yaml --lora_model_dir="/content/drive/MyDrive/DACON/Hansol_QA/models/yanolja-axolotl_v0.2"

[2024-03-09 08:06:25,626] [INFO] [numexpr.utils._init_num_threads:160] [PID:3657] NumExpr defaulting to 2 threads.
[2024-03-09 08:06:25,944] [INFO] [datasets.<module>:58] [PID:3657] PyTorch version 2.1.1 available.
[2024-03-09 08:06:25,945] [INFO] [datasets.<module>:95] [PID:3657] TensorFlow version 2.15.0 available.
[2024-03-09 08:06:25,946] [INFO] [datasets.<module>:108] [PID:3657] JAX version 0.4.23 available.
2024-03-09 08:06:28.014528: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-09 08:06:28.014578: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-09 08:06:28.016077: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for

In [ ]:
%%shell
git clone https://github.com/ggerganov/llama.cpp

cd llama.cpp

pip install -r requirements.txt
make LLAMA_OPENBLAS=1

Cloning into 'llama.cpp'...
remote: Enumerating objects: 19904, done.
remote: Total 19904 (delta 0), reused 0 (delta 0), pack-reused 19904
Receiving objects: 100% (19904/19904), 23.79 MiB | 16.10 MiB/s, done.
Resolving deltas: 100% (13978/13978), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 31.1 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.14.0 requires protobuf<4.21,>=3.20.3, but you have protobuf 4.25.3 which

In [ ]:
!python llama.cpp/convert.py /content/drive/MyDrive/DACON/Hansol_QA/model/yanolja-axolotl-aug-noise-sysprompt

Loading model file /content/drive/MyDrive/DACON/Hansol_QA/model/yanolja-axolotl-aug-noise-sysprompt/pytorch_model-00001-of-00005.bin
Loading model file /content/drive/MyDrive/DACON/Hansol_QA/model/yanolja-axolotl-aug-noise-sysprompt/pytorch_model-00001-of-00005.bin
Loading model file /content/drive/MyDrive/DACON/Hansol_QA/model/yanolja-axolotl-aug-noise-sysprompt/pytorch_model-00002-of-00005.bin
Loading model file /content/drive/MyDrive/DACON/Hansol_QA/model/yanolja-axolotl-aug-noise-sysprompt/pytorch_model-00003-of-00005.bin
Loading model file /content/drive/MyDrive/DACON/Hansol_QA/model/yanolja-axolotl-aug-noise-sysprompt/pytorch_model-00004-of-00005.bin
Loading model file /content/drive/MyDrive/DACON/Hansol_QA/model/yanolja-axolotl-aug-noise-sysprompt/pytorch_model-00005-of-00005.bin
params = Params(n_vocab=40960, n_embd=4096, n_layer=48, n_ctx=4096, n_ff=14336, n_head=32, n_head_kv=8, n_experts=None, n_experts_used=None, f_norm_eps=1e-05, rope_scaling_type=None, f_rope_freq_base=10

In [ ]:
model_dir = "/content/drive/MyDrive/DACON/Hansol_QA/model"
model_name = "yanolja-axolotl-aug-noise-sysprompt"
quant_opt = "q8_0"

# 명령어 실행
!llama.cpp/quantize "{model_dir}/{model_name}/ggml-model-f16.gguf" \
                    "{model_dir}/{model_name}/ggml-model-{quant_opt}.gguf" \
                    {quant_opt}


main: build = 2360 (6cdabe65)
main: built with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu
main: quantizing '/content/drive/MyDrive/DACON/Hansol_QA/model/yanolja-axolotl-aug-noise-sysprompt/ggml-model-f16.gguf' to '/content/drive/MyDrive/DACON/Hansol_QA/model/yanolja-axolotl-aug-noise-sysprompt/ggml-model-q8_0.gguf' as Q8_0
llama_model_loader: loaded meta data with 22 key-value pairs and 435 tensors from /content/drive/MyDrive/DACON/Hansol_QA/model/yanolja-axolotl-aug-noise-sysprompt/ggml-model-f16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     ll

In [ ]:
model_dir = "/content/drive/MyDrive/DACON/Hansol_QA/model"
model_name = "yanolja-axolotl-aug-noise-sysprompt"
quant_opt = "q8_0"

!llama.cpp/main -m {model_dir}/{model_name}/ggml-model-{quant_opt}.gguf -n 1000 --prompt "제시된 각 질문에 대해 상세하고 명확한 설명을 제공하시오. 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?"

Log start
main: build = 2360 (6cdabe65)
main: built with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu
main: seed  = 1709884504
llama_model_loader: loaded meta data with 23 key-value pairs and 435 tensors from /content/drive/MyDrive/DACON/Hansol_QA/model/yanolja-axolotl-aug-noise-sysprompt/ggml-model-q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 48
llama_model_loader: - kv   5:                  llama.feed_f